<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1XEl5z1gl6p2m8KQb35cwJSi3grDM0WiU?usp=sharing)

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Build with Latest LLMs
- Create Custom AI Apps
- Learn from Industry Experts
- Join Innovation Community

Learn more about Our Bootcamp: https://www.buildfastwithai.com/genai-course

# Kimi K2.5: Autonomous Agents with CrewAI

This notebook demonstrates how to use the **MoonshotAI Kimi-k2.5** model (via **OpenRouter**) with the **CrewAI** framework.

**Agenda:**
1.  **Setup**: Installation and Configuration.
2.  **Raw Tool Calling**: Understanding what happens under the hood (Pure OpenAI API).
3.  **CrewAI Basics**: Agents, Tasks, and Tools.
4.  **Complex Use Case**: A "Product Analyst Crew" that uses **Tavily Search** (Real-time web search), analyzes, and visualizes product data.

## 1. Setup & Installation

In [ ]:
%pip install -U -q crewai crewai-tools langchain_openai pandas matplotlib requests beautifulsoup4 nest_asyncio python-dotenv tavily-python

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
litellm 1.81.5 requires openai>=2.8.0, but you have openai 1.83.0 which is incompatible.


In [ ]:
import os

from google.colab import userdata

# CONFIGURATION
os.environ["OPENAI_API_KEY"] = userdata.get('OPENROUTER_API_KEY')
os.environ["OPENROUTER_API_KEY"] = userdata.get('OPENROUTER_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
os.environ["OPENAI_BASE_URL"] = "https://openrouter.ai/api/v1"
MODEL_NAME = "openrouter/moonshotai/kimi-k2.5" # 'openai/' prefix tells CrewAI/LiteLLM it's OpenAI-compatible

## 3. CrewAI Basics

Let's move to the framework. We defined our LLM once, and now we create an `Agent` and a `Task`.

In [ ]:
!pip install -U litellm


  Using cached openai-2.16.0-py3-none-any.whl.metadata (29 kB)
Using cached openai-2.16.0-py3-none-any.whl (1.1 MB)
  Attempting uninstall: openai
    Found existing installation: openai 1.83.0
    Uninstalling openai-1.83.0:
      Successfully uninstalled openai-1.83.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.3.23 requires openai<2.0.0,>=1.68.2, but you have openai 2.16.0 which is incompatible.
instructor 1.12.0 requires openai<2.0.0,>=1.70.0, but you have openai 2.16.0 which is incompatible.
crewai 1.9.2 requires openai~=1.83.0, but you have openai 2.16.0 which is incompatible.


In [ ]:
from crewai import Agent, Task, Crew, LLM
from crewai.tools import tool

OPENROUTER_API_KEY=os.environ.get("OPENROUTER_API_KEY")
# Define the custom LLM
kimi_llm = LLM(
    model=MODEL_NAME,
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)


# Simple Agent
translator = Agent(
    role='Professional Translator',
    goal='Translate text accurately to French.',
    backstory='You are a linguistic expert proficient in many languages.',
    llm=kimi_llm,
    verbose=True
)

task1 = Task(
    description='Translate the following text to French: "Generative AI is transforming the world."',
    expected_output='The sentence translated into French.',
    agent=translator
)

crew = Crew(agents=[translator], tasks=[task1], verbose=True)
result = crew.kickoff()
print("\n Translation Result:", result)

## 4. Complex Use Case: Product Analyst Crew

We will build a crew with three specialized agents:
1.  **Researcher**: Searches the live web using **Tavily** for product details and prices.
2.  **Analyst**: Analyzes the product and creates a Pros/Cons table.
3.  **Visualizer**: Creates a chart.

### Step 4.1: Define Tools
We use `crewai_tools.TavilySearchResults` for real search.

In [ ]:
import random
import pandas as pd
import matplotlib.pyplot as plt
from crewai_tools import TavilySearchTool

# Initialize Tavily Search Tool (requires TAVILY_API_KEY env var)
search_tool = TavilySearchTool(
    max_results=5,            # Number of results to return
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=False
)


@tool("Chart Generator")
def generate_price_chart(prices_str: str) -> str:
    """Accepts a string dictionary of prices and generates a bar chart. Example input: "{'Amazon': 500, 'BestBuy': 520}" """
    try:
        prices = eval(prices_str)
        df = pd.DataFrame(list(prices.items()), columns=['Store', 'Price'])

        plt.figure(figsize=(8, 5))
        plt.bar(df['Store'], df['Price'], color=['#ff9999', '#66b3ff', '#99ff99', '#ffcc99'])
        plt.title('Market Price Comparison')
        plt.xlabel('Store')
        plt.ylabel('Price ($)')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.savefig('price_chart.png') # Save to file so we don't block
        plt.show()
        return "Chart generated and saved as price_chart.png"
    except Exception as e:
        return f"Failed to plot: {e}"


### Step 4.2: Define Agents

In [ ]:
researcher = Agent(
    role='Tech Researcher',
    goal='Find accurate and current market prices for products using web search.',
    backstory='You are a meticulous researcher who checks multiple sources on the web to find the best deals.',
    tools=[search_tool], # Using Tavily
    llm=kimi_llm,
    verbose=True
)

analyst = Agent(
    role='Product Analyst',
    goal='Synthesize data into clear descriptions and comparison tables.',
    backstory='You are an expert at summarizing tech products for consumers.',
    llm=kimi_llm,
    verbose=True
)

visualizer = Agent(
    role='Data Visualizer',
    goal='Create visual charts from data.',
    backstory='You love turning numbers into beautiful graphs.',
    tools=[generate_price_chart],
    llm=kimi_llm,
    verbose=True
)

### Step 4.3: Define Tasks & Run Crew

Note: Ensure your `TAVILY_API_KEY` is set for the researcher to work.

In [ ]:
product_name = "Google Pixel 9 Pro"

task_research = Task(
    description=f'Search the web for the current market prices of {product_name} at major retailers (Amazon, BestBuy, etc.). Create a python dictionary string of the prices format like "{{\'Store\': Price}}". Also get a brief technical summary.',
    expected_output='A Python dictionary string of prices and a 3-sentence summary.',
    agent=researcher
)

task_visualize = Task(
    description='Using the prices found by the Researcher, generate a bar chart.',
    expected_output='Confirmation that the chart was generated.',
    agent=visualizer,
    context=[task_research] # This task waits for research
)

task_analyze = Task(
    description=f'Create a comprehensive report for {product_name}. Include the summary, a Markdown table of Pros/Cons, and mention the price trend.',
    expected_output='A full markdown report with tables and description.',
    agent=analyst,
    context=[task_research, task_visualize]
)

crew = Crew(
    agents=[researcher, visualizer, analyst],
    tasks=[task_research, task_visualize, task_analyze],
    verbose=True
)

result = crew.kickoff()

print("\n########################\n")
print("## FINAL CREW REPORT ##\n")
print(result)

╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  3740d60b-8424-418f-b294-b26edf99d9ec                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: Search the web for the current market prices of Google Pixel 9 Pro at major retailers (Amazon, BestBuy,  │
│  etc.). Create a python dictionary string of the prices format like "{'Store': Price}". Also get a brief        │
│  technical summary.                                                                                             │
│  ID: a3453722-eb7f-46c9-9755-3a7e675de01b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tech Researcher                                                                                         │
│                                                                                                                 │
│  Task: Search the web for the current market prices of Google Pixel 9 Pro at major retailers (Amazon, BestBuy,  │
│  etc.). Create a python dictionary string of the prices format like "{'Store': Price}". Also get a brief        │
│  technical summary.                                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── 🔧 Tool Execution Started (#1) ─────────────────────────────────────────╮
│                                                                                                                 │
│  Tool: tavily_search                                                                                            │
│  Args: {"query": "Google Pixel 9 Pro price Amazon BestBuy current market price 2024"}                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "query": "Google Pixel 9 Pro price Amazon BestBuy current market price 2024",                                │
│    "follow_up_questions": null,                                                                                 │
│    "answer": "The Google Pixel 9 Pro is priced at $789.37 on Amazon as of January 2026. Best Buy lists it for   │
│  $1,258.03. The price fluctuates based on promotions and seller.",                                              │
│    "images": [],                                                                                                │
│    "results": [                                                                                                 │
│      {                                                                                                          │
│        "url": "https://camelcamelcamel.com/product/B0D7HXM4HW",                                                 │
│        "title": "Google Pixel 9 Pro - Obsidian - 256 GB - CamelCamelCamel",                                     │
│        "content": "Image 4: Amazon Price History Chart\n\n| Price Type | Lowest Ever | Highest Ever | Current   │
│  + | Average |\n ---  --- \n| Amazon | $749.00 (Nov 23, 2025) | $1,244.50 (Aug 18, 2025)  $982.33 |\n| 3rd      │
│  Party New | $943.99 (Dec 27, 2024) | $1,490.00 (Aug 25, 2024) | $1,079.72 (Nov 01, 2025) | $1,201.86 |\n| 3rd  │
│  Party Used | $608.44 (Oct 23, 2025) | $1,403.53 (Sep 25, 2024) | $789.37 (Jan 23, 2026) | $866.35              │
│  |\n\ncamelcamelcamel monitors individual product prices across three Price Types: Amazon, Third Party New,     │
│  and Third Party Used. \"Amazon\" means the price of an item Sold By and Shipped By Amazon itself; the two      │
│  Third Party price types refer to third party prices on Amazon in New or Used condition, usually by             │
│  marketplace sellers.\n\nShare:\n\n### Product Details [...] ASIN: B0D7HXM4HW UPC: 840353906174 EAN:            │
│  084035390617 PRODUCT GROUP: Wireless CATEGORY: Office Product MANUFACTURER: Google MODEL: GR83Y\n\nView all    │
│  sizes / colors\n\n### Amazon Price History\n\nThis is our record of Amazon price changes for Go...",           │
│        "score": 0.8660535,                                                                                      │
│        "raw_content": "Google Pixel 9 Pro - Unlocked Android Smartphone with Gemini, Triple Rear Camera         │
│  System, 24-Hour Battery, and 6.3\" Super Actua Display - Obsidian - 256 GB | Amazon price tracker / tracking,  │
│  Amazon price history charts, Amazon price watches, Amazon price drop alerts |                                  │
│  camelcamelcamel.com\n===============\n\n*   camel camel camel\n*   [Sign                                       │
│  Up](https://camelcamelcamel.com/signup \"Sign up for a Camel account!\")\n*   [Sign In](https://came...        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Product Analyst                                                                                         │
│                                                                                                                 │
│  Task: Create a comprehensive report for Google Pixel 9 Pro. Include the summary, a Markdown table of           │
│  Pros/Cons, and mention the price trend.                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Product Analyst                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Google Pixel 9 Pro: Comprehensive Market Report                                                              │
│                                                                                                                 │
│  ## Executive Summary                                                                                           │
│                                                                                                                 │
│  The Google Pixel 9 Pro represents Google's flagship smartphone offering, featuring a compact yet premium       │
│  6.3-inch Super Actua display with a fluid 120Hz refresh rate. Powered by the custom Google Tensor G4           │
│  processor paired with 16GB of RAM, the device delivers flagship-tier performance optimized for AI-driven       │
│  tasks. The camera system stands out with its pro-level triple-lens configuration, including a 50MP main        │
│  sensor, 48MP ultrawide, and 48MP telephoto lens, bolstered by advanced computational photography features      │
│  such as Add Me and Magic Editor. With IP68 water and dust resistance, 24+ hour battery life, and an            │
│  industry-leading commitment to 7 years of OS and security updates, the Pixel 9 Pro is positioned as a          │
│  long-term investment. Deep integration of Gemini AI throughout the user experience further distinguishes this  │
│  device in the competitive flagship market.                                                                     │
│                                                                                                                 │
│  ## Key Specifications                                                                                          │
│                                                                                                                 │
│  | Feature | Specification |                                                                                    │
│  |---------|--------------|                                                                                     │
│  | **Display** | 6.3-inch Super Actua, 120Hz refresh rate |                                                     │
│  | **Processor** | Google Tensor G4 |                                                                           │
│  | **RAM** | 16GB |                                                                                             │
│  | **Rear Cameras** | 50MP Main, 48MP Ultrawide, 48MP Telephoto |                                               │
│  | **Battery Life** | 24+ hours |                                                                               │
│  | **Durability** | IP68 water and dust resistance |                                                            │
│  | **Software Support** | 7 years of OS and security updates |                                                  │
│  | **AI Integration** | Gemini AI, Add Me, Magic Editor |                                                       │
│                                                                                                                 │
│  ## Pros and Cons                                                                                               │
│                                                                                                                 │
│  | Pros | Cons |                                       

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  Create a comprehensive report for Google Pixel 9 Pro. Include the summary, a Markdown table of Pros/Cons, and  │
│  mention the price trend.                                                                                       │
│  Agent:                                                                                                         │
│  Product Analyst                                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


########################

## FINAL CREW REPORT ##

# Google Pixel 9 Pro: Comprehensive Market Report

## Executive Summary

The Google Pixel 9 Pro represents Google's flagship smartphone offering, featuring a compact yet premium 6.3-inch Super Actua display with a fluid 120Hz refresh rate. Powered by the custom Google Tensor G4 processor paired with 16GB of RAM, the device delivers flagship-tier performance optimized for AI-driven tasks. The camera system stands out with its pro-level triple-lens configuration, including a 50MP main sensor, 48MP ultrawide, and 48MP telephoto lens, bolstered by advanced computational photography features such as Add Me and Magic Editor. With IP68 water and dust resistance, 24+ hour battery life, and an industry-leading commitment to 7 years of OS and security updates, the Pixel 9 Pro is positioned as a long-term investment. Deep integration of Gemini AI throughout the user experience further distinguishes this device in the competitive flagship market.

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  3740d60b-8424-418f-b294-b26edf99d9ec                                                                           │
│  Final Output: # Google Pixel 9 Pro: Comprehensive Market Report                                                │
│                                                                                                                 │
│  ## Executive Summary                                                                                           │
│                                                                                                                 │
│  The Google Pixel 9 Pro represents Google's flagship smartphone offering, featuring a compact yet premium       │
│  6.3-inch Super Actua display with a fluid 120Hz refresh rate. Powered by the custom Google Tensor G4           │
│  processor paired with 16GB of RAM, the device delivers flagship-tier performance optimized for AI-driven       │
│  tasks. The camera system stands out with its pro-level triple-lens configuration, including a 50MP main        │
│  sensor, 48MP ultrawide, and 48MP telephoto lens, bolstered by advanced computational photography features      │
│  such as Add Me and Magic Editor. With IP68 water and dust resistance, 24+ hour battery life, and an            │
│  industry-leading commitment to 7 years of OS and security updates, the Pixel 9 Pro is positioned as a          │
│  long-term investment. Deep integration of Gemini AI throughout the user experience further distinguishes this  │
│  device in the competitive flagship market.                                                                     │
│                                                                                                                 │
│  ## Key Specifications                                                                                          │
│                                                                                                                 │
│  | Feature | Specification |                                                                                    │
│  |---------|--------------|                                                                                     │
│  | **Display** | 6.3-inch Super Actua, 120Hz refresh rate |                                                     │
│  | **Processor** | Google Tensor G4 |                                                                           │
│  | **RAM** | 16GB |                                                                                             │
│  | **Rear Cameras** | 50MP Main, 48MP Ultrawide, 48MP Telephoto |                                               │
│  | **Battery Life** | 24+ hours |                                                                               │
│  | **Durability** | IP68 water and dust resistance |                                                            │
│  | **Software Support** | 7 years of OS and security updates |                                                  │
│  | **AI Integration** | Gemini AI, Add Me, Magic Editor |                                                       │
│                                                                                                                 │
│  ## Pros and Cons                                     

In [ ]:
print("## FINAL CREW REPORT ##\n")
print(result)